# Routing: Logical

![logical routing](../images/images-logical.png)

**Logical Routing** is a technique to direct queries to the most suitable data source or system based on predefined rules or conditions. It relies on **explicit criteria**, like keywords, query patterns, or metadata, to decide where a query should go.

### The Problem Logical Routing Solves
Not all queries are the same, and different types of questions may require information from different sources. Logical routing ensures that:
- Queries are handled by the most relevant system or data source.
- Simple rules can efficiently filter and guide queries without needing complex computations.

### How Logical Routing Works
1. **Define Rules**:
   - Rules are set up in advance to match specific query features to a corresponding destination.
   - Example:
     - Queries containing the word "Python" go to a database on programming languages.
     - Queries asking about "health" go to a medical knowledge base.

2. **Evaluate the Query**:
   - The system analyzes the query based on the predefined rules.
   - Example:
     - Rule: "If the query contains the keyword 'tax,' route it to the financial data source."

3. **Route the Query**:
   - Based on the rule evaluation, the query is sent to the appropriate data source or processing system.

### Simple Example
#### Query:
*"What are the top frameworks for Python development?"*

#### Logical Routing:
1. **Keyword Detection**:
   - The rule detects the keyword "Python."
2. **Apply Rule**:
   - Match: "Route to the programming frameworks database."
3. **Retrieve and Respond**:
   - The system retrieves relevant Python frameworks like Django, Flask, and FastAPI.

### Why Logical Routing Works
- **Efficiency**: Simple rules process queries quickly without the need for heavy computation.
- **Clarity**: The rules are explicit and easy to understand or modify.
- **Effectiveness**: Works well when query intent is clear and matches predefined categories.

### Key Features of Logical Routing
- **Keyword Matching**: Direct routing based on specific words or phrases in the query.
- **Pattern Recognition**: Use of patterns like question formats (e.g., "What is..." or "How to...").
- **Metadata Analysis**: Routing based on additional query data like user preferences or query source.

### Example Use Case
In a customer support system:
- **Rule 1**: Queries with the keyword "billing" are routed to the finance team.
- **Rule 2**: Queries with "password reset" go to the IT support database.
- **Rule 3**: Queries mentioning "shipping" are directed to the logistics department.

### Benefits of Logical Routing
- **Simplicity**: Easy to implement and maintain.
- **Speed**: No need for complex models or processing steps.
- **Predictability**: The routing behavior is transparent and consistent.

In short, **Logical Routing** is like a decision tree for queries: if a query fits a certain rule, it’s sent down the right path. It’s simple, fast, and works well when the rules are clear and the query categories are well-defined!

![logical routing](../images/logical_routing.png)

## Setup

In [1]:
%run "../Z - Common/setup.ipynb"

Stored 'enable_langsmith' (bool)


USER_AGENT environment variable not set, consider setting it to identify your requests.


Let's start by building a prompt and chain to route.

In [2]:
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import Literal

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )

# LLM with function call 
# Structured Outputs is a feature that ensures the model will always generate responses that adhere to your supplied JSON Schema, 
# so you don't need to worry about the model omitting a required key, or hallucinating an invalid enum value.
structured_llm = llm.with_structured_output(RouteQuery)

# Prompt 
template = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source."""

prompt_router = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        ("human", "{question}"),
    ]
)

# Define router 
chain_router = prompt_router | structured_llm

In [3]:
question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result = chain_router.invoke({"question": question})
result

RouteQuery(datasource='python_docs')

We can then use a [custom function to route between different sub chains](https://python.langchain.com/docs/how_to/routing/#using-a-custom-function-recommended) based on the response.

In [4]:
from langchain_core.runnables import RunnableLambda

def choose_route(result):
    if "python_docs" in result.datasource.lower():
        ### Logic here 
        return "chain for python_docs"
    elif "js_docs" in result.datasource.lower():
        ### Logic here 
        return "chain for js_docs"
    else:
        ### Logic here 
        return "golang_docs"


chain = chain_router | RunnableLambda(choose_route)

In [5]:
chain.invoke({"question": question})

'chain for python_docs'